# Convolutional Neural Networks

## Import libraries

In [1]:
import os
import tensorflow
os.environ['KERAS_BACKEND'] = 'tensorflow'

import wget
import numpy as np
import pandas as pd
import shutil
import zipfile
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

## Download and extract dataset

In [2]:
if os.path.exists('data'):
    print("Path already exists")
else:
    print("Creating the data directory")
    os.mkdir('data')
    print("Directory created")

Path already exists


In [3]:
data_url = 'https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/P16-Convolutional-Neural-Networks.zip'
file_name = data_url.split("/")[-1]
if not os.path.exists('data/' + file_name):
    print('Need to download dataset')
    wget.download(data_url, out = 'data')
    print('Data downloaded')
else:
    print('Dataset zip available')

Need to download dataset


KeyboardInterrupt: 

In [ ]:
from distutils.dir_util import copy_tree
with zipfile.ZipFile('data/' + file_name, 'r') as file:
    print("Extracting dataset")
    file.extractall()
    copy_tree('Convolutional_Neural_Networks/dataset/', 'data/')
    print("Dataset extracted and created")

In [ ]:
try:
    shutil.rmtree('Convolutional_Neural_Networks')
    os.remove('data/' + file_name)
    print("Extra files removed")
except:
    print("Could not delete extra files")

## Other imports

In [4]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense

Using TensorFlow backend.


## Building the model

In [18]:
classifier = Sequential()
classifier.add(Convolution2D(filters = 32, kernel_size = (3, 3), input_shape = (64, 64, 3), activation = 'relu', data_format = 'channels_last'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 62, 31, 16)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 30752)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               3936384   
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
Total params: 3,937,409
Trainable params: 3,937,409
Non-trainable params: 0
_________________________________________________________________
